In [1]:
from pyspark.sql.functions import year, month, dayofmonth, hour, col, lit, desc, coalesce, udf,when
from pyspark.sql.types import IntegerType

In [2]:
Accounts = spark.read.format('delta').load("/mnt/silver/cab/accounts")
Companies = spark.read.format('delta').load("/mnt/silver/cab/companies")
Incidents = spark.read.format('delta').load("/mnt/silver/dynamics/incidents")

In [3]:
display(Incidents)

statecode,statename,accountid,modifiedon,statuscode,statusname,incidentid,casetypecode,casetype,prioritycode,priority,severitycode,severity,createdon,resolvedon,companyid,isescalated
1,Resolved,89f81254-115c-e411-80e9-00155d007a09,2014-10-25T18:16:01.000+0000,5,Problem Solved,000387f7-725c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2008-09-25T12:24:23.000+0000,2014-10-25T18:16:01.000+0000,null,false
1,Resolved,64d95cdb-f2d5-e311-80d2-00155d002f03,2014-10-26T00:00:30.000+0000,5,Problem Solved,00044313-a35c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2012-09-10T13:25:38.000+0000,2014-10-26T00:00:30.000+0000,null,false
1,Resolved,aa2e928e-425c-e411-80e9-00155d007a09,2014-10-27T23:36:28.000+0000,5,Problem Solved,0007d621-615c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2008-01-17T16:22:19.000+0000,2014-10-27T23:36:28.000+0000,null,false
1,Resolved,f0ae8dbd-375c-e411-80e9-00155d007a09,2014-10-27T22:26:01.000+0000,5,Problem Solved,0009042e-955c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2006-12-04T20:44:04.000+0000,2014-10-27T22:26:01.000+0000,null,false
1,Resolved,7286e5b7-f932-e411-80e3-00155d007a09,2017-03-27T12:46:55.000+0000,5,Problem Solved,000be92d-eb12-e711-80f9-00155d002f2e,1,Question,2,Normal,100000002,Normal,2017-03-27T12:44:56.000+0000,2017-03-27T12:46:55.000+0000,null,false
1,Resolved,62d95cdb-f2d5-e311-80d2-00155d002f03,2016-12-07T16:43:59.000+0000,5,Problem Solved,000f6481-ab94-e611-80f4-00155d002f2e,3,Request,2,Normal,100000002,Normal,2016-10-17T20:51:47.000+0000,2016-10-17T20:57:23.000+0000,null,false
1,Resolved,bc09fd26-255c-e411-80e9-00155d007a09,2014-10-27T21:05:40.000+0000,5,Problem Solved,0010381c-745c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2005-09-19T22:17:48.000+0000,2014-10-27T21:05:40.000+0000,null,false
1,Resolved,56e3a446-1b5c-e411-80e9-00155d007a09,2014-10-25T17:15:50.000+0000,5,Problem Solved,0011678d-6a5c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2011-02-17T16:41:25.000+0000,2014-10-25T17:15:50.000+0000,null,false
1,Resolved,fddc82a7-1b5c-e411-80e9-00155d007a09,2014-10-27T22:40:45.000+0000,5,Problem Solved,00156184-9b5c-e411-80e9-00155d007a09,null,BAD DATA,2,Normal,1,BAD DATA,2007-03-01T18:45:02.000+0000,2014-10-27T22:40:45.000+0000,null,false
1,Resolved,68fc6a49-ce58-e511-8116-00155d007a09,2017-05-08T17:26:23.000+0000,5,Problem Solved,0017a554-1334-e711-80fb-00155d007a25,2,Problem,2,Normal,100000002,Normal,2017-05-08T17:25:29.000+0000,2017-05-08T17:26:23.000+0000,null,false


In [4]:
Accounts.createOrReplaceTempView('Accounts')
Incidents.createOrReplaceTempView('Incidents')

accountsWIncidents = spark.sql('''
                            SELECT 
                                a.BillingAccountId
                                ,a.AccountName
                                ,a.Status
                                ,a.IsTest
                                ,(SELECT count(1) FROM Incidents i WHERE a.accountid == i.accountid ) as IncidentCount
                            FROM Accounts a
                            where a.BillingAccountId is not null

                      ''')

In [5]:
display(accountsWIncidents.orderBy(col('IncidentCount').desc()))

BillingAccountId,AccountName,Status,IsTest,IncidentCount
4700,Connect,1,false,934
1747,Ana1,1,true,167
7560,TestAcct_Ola_5,1,true,164
4433,Coast Citrus Distributors,1,false,160
2801,ACTSOFT INC,1,true,141
7388,COLOR SPOT NURSERIES,1,false,111
468,Actsoft Technical Support,1,true,96
1104,"Andersen Commerical Plumbing, Inc.",1,false,92
2066,Appleby Systems Inc,1,false,91
2506,Central Coast Distributing,1,false,87
